<a href="https://colab.research.google.com/github/karanbhatt1/ConversationalChatBot/blob/main/ArtificalIntelligenceProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OBECTIVE:
> To build a conversational AI which takes the input data like pdf, txt,image,json and answer the questions related to it.


# Installing the useful libraries

In [4]:
!pip install langchain langchain_community langchain_chroma

  Using cached langchain_community-0.3.30-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_chroma-0.2.6-py3-none-any.whl.metadata (1.1 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached chromadb-1.1.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.2 kB)
  Using cached pybase64-1.4.2-cp312-cp312-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (8.7 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.9 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.37.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using ca

In [5]:
!pip install chromadb

In [6]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 9.7 MB/s eta 0:00:00


# Importing the dependencies

In [7]:
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from pypdf import PdfReader
import re

# Step 2 -- Document Processing

## 1. Taking a plain text file from user
> Merging the text in the document.

1-> **!pip install pypdf [to handle pdf files]**
</br>
2-> **from pypdf import PdfReader**
</br>
3-> **readerObject.pages** to get toatal pages in pdf
<br>
4-> **.extract_text()**: to extract text from particular page

In [65]:
fldr = "/content/drive/MyDrive/machinelearningprojects/AIProject/NIPS-2017-attention-is-all-you-need-Paper.pdf"

In [66]:
pdfFile = PdfReader(fldr)

In [67]:
def clean_text(text):
    # Replace multiple spaces/tabs/newlines with a single space
    text = re.sub(r'[\t\n\r]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [68]:
len(pdfFile.pages)

11

In [69]:
# concatinating the text:
combinedText = ""
for page in pdfFile.pages:
  txt = page.extract_text()
  if txt:
    clntext = clean_text(txt)
    combinedText += clntext+"/n"

In [70]:
combinedText

'Attention Is All You Need Ashish Vaswani⇤ Google Brain avaswani@google.com Noam Shazeer⇤ Google Brain noam@google.com Niki Parmar⇤ Google Research nikip@google.com Jakob Uszkoreit⇤ Google Research usz@google.com Llion Jones⇤ Google Research llion@google.com Aidan N. Gomez⇤ † University of Toronto aidan@cs.toronto.edu Łukasz Kaiser⇤ Google Brain lukaszkaiser@google.com Illia Polosukhin⇤ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring signiﬁcantly less time to train. Our mode

In [71]:
len(combinedText) # 461134

32635

# Step 3 Splitting the text
> The data is splited to chunks and converted to document using:
> .create_document([cleaned_text])

> Total chunks = 119 with size of each chunk as 5000

In [72]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [73]:
len(combinedText) # len = four lakhs sixty one thousand one hundred thirty four token or text

32635

In [74]:
def chunk_size(combinedText):
  if(len(combinedText)>250000):
    return 70000
  elif(len(combinedText)>100000):
    return 10000
  elif(len(combinedText)>50000):
    return 5000
  elif(len(combinedText)>25000):
    return 1000
  else:
    return 500

In [75]:
splitter = CharacterTextSplitter(separator='/n',
                                 chunk_size= 5000,
                                 chunk_overlap = 50,
                                 length_function = len
                                 )

In [76]:
splited_chunks = splitter.split_text(combinedText) # This can be used for small documents. Seprating based on seprator.

In [77]:
splited_chunks

['Attention Is All You Need Ashish Vaswani⇤ Google Brain avaswani@google.com Noam Shazeer⇤ Google Brain noam@google.com Niki Parmar⇤ Google Research nikip@google.com Jakob Uszkoreit⇤ Google Research usz@google.com Llion Jones⇤ Google Research llion@google.com Aidan N. Gomez⇤ † University of Toronto aidan@cs.toronto.edu Łukasz Kaiser⇤ Google Brain lukaszkaiser@google.com Illia Polosukhin⇤ ‡ illia.polosukhin@gmail.com Abstract The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring signiﬁcantly less time to train. Our mod

In [78]:
len(splited_chunks)

10

In [79]:
stxt = splitter.create_documents([combinedText])

In [80]:
len(stxt) # 119 chunks

10

## Using RecursiveCharacterTextSplitter
> it is used to make chunks based on size not dependent on seprator.


In [158]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,    # max 1000 characters per chunk
    chunk_overlap=10   # 200 characters overlap
)
chunks = splitter.split_text(combinedText)
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 136


In [159]:
finalDocs = splitter.create_documents([combinedText])

In [160]:
len(finalDocs)

136

# Step 3 : Import the  Embedding models
> For contextual embedding we can use any embedding model
> BERT, GPT , HuggingFace Embedding, OpenAI EMbedings

In [161]:
import chromadb

In [162]:
from sentence_transformers import SentenceTransformer

In [163]:
from langchain.embeddings  import HuggingFaceEmbeddings

In [164]:
hf_model = HuggingFaceEmbeddings(model_name  = "sentence-transformers/all-MiniLM-L6-v2")

# Step 4 Making Database and adding chunks in collection
> we are using the langchain wrapped chromadb
> <br> It provide the functions to add the data in the database and return unique id.

**vdb = Chroma(collection_name="chkdatacoll" ,embedding_function=hf_model)**

In [165]:
from langchain.schema import Document
from langchain_community.vectorstores import Chroma

# LangChain-managed Chroma vectorstore

In [166]:
# clt.list_collections()
# clt.delete_collection("name of the database")
# clt.delete_collection("chkdatacoll")
# clt.create_collection(name = ,
# embedding_function = )

In [167]:
vdb = Chroma(collection_name="chkdatacoll" ,embedding_function=hf_model)

In [168]:
stgids = vdb.add_documents(finalDocs) # This is used to fetch the ids from the chunk of the document.

In [169]:
for doc, sid in zip(finalDocs, stgids):
    doc.metadata["storage_id"] = sid

# Similarity Search: RAG
> RAG works on the basis of Similarity search.
> res = vdb.similarity_search( query = "" , k= no of ansers or data you want)

In [170]:
res = vdb.similarity_search(
    query = "How to build a new habbit?",
    k = 1
)

In [171]:
res[0].page_content

'models on the English-to-German and English-to-French newstest2014 tests at a fraction of the training cost. Model BLEU Training Cost (FLOPs) EN-DE EN-FR EN-DE EN-FR ByteNet [15] 23.75 Deep-Att + PosUnk [32] 39.2 1.0 · 1020 GNMT + RL [31] 24.6 39.92 2.3 · 1019 1.4 · 1020 ConvS2S [8] 25.16 40.46 9.6 · 1018 1.5 · 1020 MoE [26] 26.03 40.56 2.0 · 1019 1.2 · 1020 Deep-Att + PosUnk Ensemble [32] 40.4 8.0 · 1020 GNMT + RL Ensemble [31] 26.30 41.16 1.8 · 1020 1.1 · 1021 ConvS2S Ensemble [8] 26.36 41.29'

# STARTING WITH RAG:

### R --> Retriever retrive the chunks from the vectordb.

## a. Create a retriver

In [172]:
retriver = vdb.as_retriever() #  A fun to to use the vector database as retriver

## B. LLM Instance
    > 1. Using the "Llama-Mimi-8B" model

In [173]:
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline

In [174]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

 #### This dosen't have padding inbuit so we do the manual padding.

In [175]:
google_tokenizer_1 = AutoTokenizer.from_pretrained("google/flan-t5-base")
google_model_1 = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [176]:
if google_tokenizer_1.pad_token is None:
     google_tokenizer_1.add_special_tokens({'pad_token':'[PAD]'})#google_tokenizer_1.eos_token})

In [177]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [178]:
google_model_1.resize_token_embeddings(len(google_tokenizer_1))

Embedding(32100, 768)

In [179]:
model.config.pad_token_id = google_tokenizer_1.pad_token_id

#### DeepSeek Model

In [180]:
deepseek_model =  AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-V3.2-Exp")
# model = AutoModelForCausalLM.from_pretrained("llm-jp/Llama-Mimi-8B")

# STEP
## Creating generator using pipeline

In [181]:
genrator = pipeline("text2text-generation",
                    model = google_model_1,
                    tokenizer = google_tokenizer_1,
                    max_length = 10000,
                    max_new_tokens = 150)

Device set to use cuda:0


In [182]:
from langchain.llms import HuggingFacePipeline

>Transforming to the HuggingFace Pipeline to ensure the embeddings and
LLM.

In [183]:
llm = HuggingFacePipeline(pipeline = genrator)

## c. Design a Prompt

In [184]:
template = """ Use the context provided to answer the question. if you don't know then answer, say This is out of context.

            Context:
            {context}
            Question: {question}
            Answer:
            """

In [185]:
custom_template = PromptTemplate(
    template = template,
    input_variables = ["context", "question"]
)

In [186]:
custom_template

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=" Use the context provided to answer the question. if you don't know then answer, say This is out of context.\n\n            Context: \n            {context}\n            Question: {question}\n            Answer:\n            ")

# Now Connection all things using the RAG chain

In [187]:
rag_chain = (
    {"context": retriver, "question": RunnablePassthrough()} # used to take input from user
    | custom_template
    | llm
    | StrOutputParser() # Format output in a presentable format
)

# Flow:
> Input Question <br>
> Retriver -> Format -> Context<br>
> {Context, question}<br>
> Prompt Template (custom_template)<br>
> LLM (Generate answer)<br>
> StrOutputParser (final otuput string)

# Test

In [188]:
querry = "what is encoder?"

In [189]:
res = rag_chain.invoke(querry)

In [190]:
res

'a stack ofN =6 identical layers'